<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<p style="font-size:48px;line-height:1.5;">Towards brain builders modularization</p>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

<br/>
<br/>
# Context
<img src="assets/BrainBuilder.svg" width="100%">
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

<br/>
<br/>
# Implementation PoC
<br/>
<ul style="font-size:18px">
    <li style="margin:1em"> A first step towards a framework. </li>
    <li style="margin:1em"> It builds collections of cells and give them properties based on volumetric data. </li>
    <li style="margin:1em"> Reconstructed morphologies. </li>
    <li style="margin:1em"> Work in progress! </li>
</ul>
<img src="assets/Modules_all.svg" width="100%">
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

In [ ]:
annotation = gb.VoxelData.load_metaio("../data/P56_Mouse_annotation/annotation.mhd")
hierarchy = gb.load_hierarchy("../data/P56_Mouse_annotation/annotation_hierarchy.json")['msg'][0]
full_density = gb.VoxelData.load_metaio("../data/atlasVolume/atlasVolume.mhd")

region_name = "Primary somatosensory area"


### Execution

In [ ]:
from brainbuilder.select_region import select_region, select_hemisphere

density = select_region(annotation.raw, full_density, hierarchy, region_name)
density.raw = select_hemisphere(density.raw, left=True)

### Output

In [ ]:
notebook.show_volume('density', density)

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

## Positions
<img src="assets/Modules_2.svg" width="75%" align="left">

### Inputs

In [ ]:
voxel_dimensions = full_density.voxel_dimensions
total_cell_count = 200000

### Execution

In [ ]:
from brainbuilder.cell_positioning import cell_positioning

cells.positions = cell_positioning(density, total_cell_count)

### Output

In [ ]:
notebook.show_points('positions', cells)

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

## Excitatory / Inhibitory
<img src="assets/Modules_3.svg" width="75%" align="left">

### Inputs

In [ ]:
recipe_filename = "../data/bbp_recipe/builderRecipeAllPathways.xml"
recipe_sdist = bbp.load_recipe_as_spatial_distribution(recipe_filename, annotation, hierarchy, region_name)
sclass_sdist = recipe_sdist.reduce_distribution_collection('synapse_class')

### Execution

In [ ]:
from brainbuilder.assignment_synapse_class import assign_synapse_class_from_spatial_dist
chosen_synapse_class = assign_synapse_class_from_spatial_dist(cells.positions, sclass_sdist)
cells.add_properties(chosen_synapse_class)

### Output

In [ ]:
notebook.show_property('synapse_class', cells)

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

## M-Type and E-Type
<img src="assets/Modules_4.svg" width="75%" align="left">

### Execution

In [ ]:
chosen_me = assign_metype(cells.positions, cells.properties.synapse_class, recipe_sdist)
cells.add_properties(chosen_me)

### Outputs

In [ ]:
from brainbuilder.assignment_metype import assign_metype
notebook.show_property('mtype', cells)

In [ ]:
notebook.show_property('etype', cells)

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

## Reconstructed Morphology
<img src="assets/Modules_5.svg" width="75%" align="left">

### Inputs

In [ ]:
neurondb_filename = "../data/bbp_recipe/neurondb.dat"
neuron_sdist = bbp.load_neurondb_v4_as_spatial_distribution(neurondb_filename, annotation, hierarchy, region_name, percentile=0.92)

### Execution

In [ ]:
from brainbuilder.assignment_morphology import assign_morphology
chosen_morphology = assign_morphology(cells.positions, cells.properties[['mtype', 'etype']], neuron_sdist)
cells.add_properties(chosen_morphology)

### Output

In [ ]:
notebook.show_property('morphology', cells)

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

## Orientation
<img src="assets/Modules_6.svg" width="75%" align="left">

### Inputs

In [ ]:
from brainbuilder.orientation_field_sscx import compute_orientation_field
from brainbuilder.utils import vector_fields as vf
orientation_field = compute_orientation_field(annotation, hierarchy, region_name)
notebook.show_vectors('up', vf.split_orientation_field(orientation_field)[1], 7000, voxel_dimensions)

### Execution

In [ ]:
from brainbuilder.assignment_orientation import assign_orientations
cells.orientations = assign_orientations(cells.positions, orientation_field, voxel_dimensions)

### Output

In [ ]:
notebook.show_placement('placement', cells)

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

# The Future

<ul style="font-size:18px">
  <li style="margin:1em"> This was one of many possible implementations of many possible workflows. </li>
  <li style="margin:1em"> Functionality like this will be exposed through <em>Collaboratory</em>. </li>
  <li style="margin:1em"> The modules that compose the workflow will be swappable. </li>
  <li style="margin:1em"> Work in progress! </li>
</ul>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>